In [1]:
import glob
import os
import subprocess
import whisper
import logging
import csv

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import pandas as pd
import re
import string
import ipywidgets as widgets
import os

from rake_nltk import Rake
from IPython.display import display, HTML, clear_output
from IPython.core.display import Javascript

import warnings
warnings.filterwarnings('ignore')


Extract filename from videos

In [2]:
# Get all .mp4 files in the 'videos' folder
mp4_files = glob.glob("videos/*.mp4")

In [ ]:
# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

class VideoTranscriber:
    def __init__(self, model_name="base"):
        """Initializes the VideoTranscriber with a Whisper model."""
        self.model = whisper.load_model(model_name)
        logging.info(f"Loaded Whisper model: {model_name}")

    def extract_audio(self, video_path, audio_path):
        """Extracts audio from a video file using ffmpeg."""
        try:
            command = [
                'ffmpeg', '-i', video_path, '-q:a', '0', '-map', 'a', audio_path, '-y'
            ]
            subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
            logging.info(f"Extracted audio from {video_path} to {audio_path}")
            return audio_path
        except subprocess.CalledProcessError as e:
            logging.error(f"Failed to extract audio from {video_path}: {e}")
            return None

    def transcribe_audio(self, audio_path):
        """Transcribes an audio file using Whisper."""
        try:
            result = self.model.transcribe(audio_path)
            logging.info(f"Transcribed audio from {audio_path}")
            return result["text"]
        except Exception as e:
            logging.error(f"Failed to transcribe audio from {audio_path}: {e}")
            return None

    def process_video(self, video_path):
        """Processes a video file by extracting audio and transcribing it."""
        audio_path = video_path.rsplit('.', 1)[0] + ".mp3"
        if self.extract_audio(video_path, audio_path):
            transcript = self.transcribe_audio(audio_path)
            if transcript:
                return transcript
        return None

def main():
    video_dir = "videos"  # Directory containing video files
    transcriber = VideoTranscriber(model_name="base")
    transcripts = {}

    # Process each video file in the directory
    for video_file in os.listdir(video_dir):
        if video_file.endswith(('.mp4', '.mkv', '.avi', '.mov')):  # Add more formats if needed
            video_path = os.path.join(video_dir, video_file)
            logging.info(f"Processing {video_file}...")
            transcript = transcriber.process_video(video_path)
            if transcript:
                transcripts[video_file] = transcript
            else:
                logging.warning(f"Skipped {video_file} due to errors.")

    # Save all transcripts to a text file
    with open("transcriptions.txt", "w") as f:
        for video, text in transcripts.items():
            f.write(f"{video}\n{text}\n\n")

    logging.info("Transcription complete. Saved to transcriptions.txt.")

transcription

In [ ]:
main()

In [ ]:
# Path to the input text file and output CSV file
input_file = "transcriptions.txt"
output_file = "transcriptions.csv"

# Open the input text file and output CSV file
with open(input_file, "r", encoding="utf-8") as txt_file, open(output_file, mode="w", newline="", encoding="utf-8") as csv_file:
    writer = csv.writer(csv_file)

    # Write the header row
    writer.writerow(["video", "transcription"])

    # Read the text file line by line
    video_name = None
    transcription = []

    for line in txt_file:
        line = line.strip()  # Remove leading/trailing whitespace

        # Check if the line is a video name
        if line.endswith((".mp4", ".mkv", ".avi", ".mov")):
            # If we have a previous video, write it to the CSV
            if video_name and transcription:
                writer.writerow([video_name, " ".join(transcription)])
                transcription = []  # Reset transcription for the next video

            # Set the current video name
            video_name = line

        # Otherwise, add the line to the transcription
        else:
            transcription.append(line)

    # Write the last video and transcription to the CSV
    if video_name and transcription:
        writer.writerow([video_name, " ".join(transcription)])

print(f"✅ Converted {input_file} to {output_file}")

## 3. Normalizing the text

In [ ]:
# Download NLTK data (run once)
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download('punkt_tab')

nltk.download("averaged_perceptron_tagger")  # For POS tagging

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Normalization function
def normalize_text(text):
    """
    Normalizes text by:
    1. Converting to lowercase
    2. Removing punctuation
    3. Tokenizing
    4. Removing stopwords
    5. Lemmatizing words
    6. Removing extra whitespace
    """
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords and lemmatize
    stop_words = set(stopwords.words("english"))
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    # Join tokens back into a string
    text = " ".join(tokens)

    # Remove extra whitespace
    text = " ".join(text.split())

    return text

# Path to the input CSV file and output CSV file
input_csv = "transcriptions.csv"
output_csv = "normalized_transcriptions.csv"

# Read the CSV file
df = pd.read_csv(input_csv)

# Normalize the 'transcription' column
df["transcription"] = df["transcription"].apply(normalize_text)

# Save the normalized data to a new CSV file
df.to_csv(output_csv, index=False)

print(f'Normalized input!')

## 4. Extracting key phrases and topics

In [ ]:
# Load the normalized transcriptions
df = pd.read_csv("normalized_transcriptions.csv")

# Initialize RAKE
rake = Rake()

# Extract key phrases for each transcription
df["key_phrases"] = df["transcription"].apply(
    lambda text: rake.extract_keywords_from_text(text) or rake.get_ranked_phrases()
)

# Save the updated CSV file
df.to_csv("transcriptions_with_key_phrases.csv", index=False)

## 5. Creating the dashboard

In [19]:
# Load the transcription data
try:
    df = pd.read_csv('transcriptions_with_key_phrases.csv')
    # If key_phrases is stored as string, convert to list
    if isinstance(df['key_phrases'].iloc[0], str):
        df['key_phrases'] = df['key_phrases'].apply(eval)
except:
   print('Failed to load transcriptions with key_phrases.csv')

# Create search box
search_box = widgets.Text(
    placeholder='Type to search videos...',
    layout=widgets.Layout(width='80%')
)

# Create output area for results
results_output = widgets.Output()

# Search function
def search_videos(query):
    if not query:
        return df

    query = query.lower()
    results = df[
        df['video'].str.lower().str.contains(query) |
        df['transcription'].str.lower().str.contains(query) |
        df['key_phrases'].apply(lambda x: any(query in phrase.lower() for phrase in x))
        ]
    return results

# Display function
def show_results(query):
    with results_output:
        clear_output()
        results = search_videos(query)

        if len(results) == 0:
            print("No videos found matching your search.")
            return

        print(f"Found {len(results)} videos:\n")

        for _, row in results.iterrows():
            # Show video path (assuming videos are in a 'videos' folder)
            video_path = os.path.join('videos', row['video'])
            print(f"Video: {row['video']}")
            print(f"Path: {video_path}")
            print(f"Transcription: {row['transcription'][:100]}...")  # Show first 100 chars
            print(f"Key phrases: {', '.join(row['key_phrases'][:3])}")  # Show first 3 phrases
            print("-" * 50 + "\n")


# Set up search handler
def on_search_change(change):
    show_results(change['new'])

search_box.observe(on_search_change, names='value')

# Initial display
display(widgets.HBox([widgets.Label("Search:"), search_box]))
display(results_output)

# Show all videos initially
show_results("")

Output()

In [4]:
display(HTML(f"""
<video width="400" controls>
  <source src="videos/Mod02_Intro.mp4" type="video/mp4">
  Your browser doesn't support HTML5 video
</video>
"""))